In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import make_scorer
from tensorflow import keras
from kerastuner import RandomSearch

C:\Users\PcR\AppData\Local\Temp\ipykernel_5736\3118260604.py:25: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


# Objetivo
En Red con max ya se logró una red que predice los valores máximos de longitud de onda e índice de absorción. Esta red no nada más predecirá elñ punto máximo sinó que buscaremos que prediga todo el espectro de longitudes de onda, es decir de 300 a 900nm. Como resultado esperamos una gráfica

In [2]:
files = [file for file in os.listdir('./parametros')]

param = pd.DataFrame()
for file in files:
  df = pd.read_csv('./parametros/'+file)
  param = pd.concat([param, df], axis= 0)

Dividir los datos en características (X) y etiquetas (y)

In [3]:
X = df.drop('Índice de absorción', axis=1).values
y = df['Índice de absorción'].values

Normalizar los datos usando MinMaxScaler


scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
y_normalized = scaler.fit_transform(y.reshape(-1, 1))

In [4]:
scaler = MinMaxScaler()
X_normalized = scaler.fit(X)
X_normalized = scaler.transform(X)

Dividir los datos en conjuntos de entrenamiento y prueba

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.3, random_state=42)

Intento con Keras Tuner

In [8]:
def build_model(hp):
    model = keras.Sequential()
    
    # Capa de entrada
    model.add(keras.layers.Dense(units=hp.Int('input_units', min_value=4, max_value=128, step=32),
                                 input_dim=4, 
                                 kernel_initializer='normal', 
                                 activation='relu'))
    
    # Capa oculta
    model.add(keras.layers.Dense(units=hp.Int('hidden_units', min_value=4, max_value=128, step=32),
                                 kernel_initializer='normal', 
                                 activation='relu'))
    
    # Capa de salida
    model.add(keras.layers.Dense(1, kernel_initializer='normal', activation='relu'))
    
    # Compilación
    optimizer = keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    return model

# Definir el buscador
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,  # Número de combinaciones de hiperparámetros para probar
    executions_per_trial=3,  # Número de veces que se entrena cada modelo
    directory='my_dir',
    project_name='helloworld')

# Cuando estés listo para buscar los mejores hiperparámetros, ejecuta:
tuner.search(X_train, y_train, epochs=20, validation_data=(X_test, y_test))


Trial 5 Complete [00h 00m 39s]
val_loss: 8.604404418595853e-07

Best val_loss So Far: 4.578553169191461e-07
Total elapsed time: 00h 03m 16s
INFO:tensorflow:Oracle triggered exit


Intento con GridSearch

In [241]:
# Function to generate Deep ANN model 
def make_regression_ann(Optimizer_trial):    
    model = Sequential()
    model.add(Dense(units=5, input_dim=4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=64, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='relu'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

###########################################


# Listing all the parameters to try
Parameter_Trials={'batch_size':[10],'epochs':[50,200],'Optimizer_trial':['adam'], 'validation_split' : [0.05, 0.1, 0.06,0.07]}

# Creating the regression ANN model
RegModel=KerasRegressor(make_regression_ann, verbose=0)

###########################################
from sklearn.metrics import make_scorer

# Defining a custom function to calculate accuracy
def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring=make_scorer(Accuracy_Score, greater_is_better=True)

#########################################
# Creating the Grid search space
# See different scoring methods by using sklearn.metrics.SCORERS.keys()
grid_search=GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

#########################################
# Measuring how much time it took to find the best params
import time
StartTime=time.time()

# Running Grid Search for different paramenters
grid_search.fit(X,y, verbose=1)

EndTime=time.time()
print("########## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes')

print('### Printing Best parameters ###')
grid_search.best_params_